In [1]:
import numpy as np
import pandas as pd
from numba import jit, vectorize, float64, types, int64
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm


In [2]:
# Coupling formula
mu_0 = 12.5663706127e-7
h = 6.62607015e-34

# gammas from https://kodu.ut.ee/~laurit/AK2/NMR_tables_Bruker2012.pdf
alpha_w = mu_0/4/np.pi * (6.7282860e7)**2 *(h/2/np.pi)**2

a = 3.5668e-10 # In meters
# Lattice parameters:
lattice_x = np.array([a, a, 0])/2
lattice_y = np.array([0, a, a])/2
lattice_z = np.array([a, 0, a])/2

# sites interstitiels
lattice_s = a * np.array([
    [0,0,0],
    [0.25, 0.25,0.25],
])

site_nb = lattice_s.shape[0]


@vectorize([float64(float64,float64,float64)])
def compute_spatial_coupling(x, y, z):
    norm = (x**2 + y**2 + z**2)**0.5
    return np.abs(alpha_w/norm**3*(1-3*((x+y+z))**2/norm**2/3))/2/h

@vectorize([float64(int64, int64, int64, int64, int64)])
def compute_coupling(x, y, z, a, b):
    vec = x*lattice_x + y*lattice_y + z*lattice_z + lattice_s[a] - lattice_s[b]
    return compute_spatial_coupling(vec[0], vec[1],  vec[2])

@jit
def couplings(sites):
    n = sites.shape[0]
    couplings = np.full((n , n), np.nan)
    for i in range(sites.shape[0]-1):
        for j in range(i+1, sites.shape[0]):
            x = sites[i][0] - sites[j][0]
            y = sites[i][1] - sites[j][1]
            z = sites[i][2] - sites[j][2]
            couplings[i, j] = compute_coupling(x, y, z, sites[i][3], sites[j][3])
    return couplings

@jit
def spatial_couplings(sites):
    full_sites = sites[None,:,:] - sites[:,None,:]
    couplings = compute_spatial_coupling(full_sites[:,:,0], full_sites[:,:,1], full_sites[:,:,2])
    for i in range(couplings.shape[0]):
        for j in range(0, i):
            couplings[i,j] = np.nan
    return couplings



In [3]:
c = set()
for i in range(len(lattice_s)):
    for j in range(len(lattice_s)):
        for k in range(10):
            for l in range(10):
                for m in range(10):
                    v = lattice_s[i] - lattice_s[j] + (k-5)*lattice_x + (l-5)*lattice_y + (m-5)*lattice_z
                    if np.isclose(np.linalg.norm(v), 3.88683019e-10, atol=0, rtol=1e-1):
                        print(v, i, j, k-5, l-5, m-5, compute_spatial_coupling(v[0], v[1], v[2]), np.linalg.norm(v))
np.linalg.norm([2.52e-10,2.91e-10, -0.51e-10])

[ 0.0000e+00 -3.5668e-10  0.0000e+00] 0 0 -1 -1 1 0.0 3.5668e-10
[-3.5668e-10  0.0000e+00  0.0000e+00] 0 0 -1 1 -1 0.0 3.5668e-10
[0.0000e+00 0.0000e+00 3.5668e-10] 0 0 -1 1 1 0.0 3.5668e-10
[ 0.0000e+00  0.0000e+00 -3.5668e-10] 0 0 1 -1 -1 0.0 3.5668e-10
[3.5668e-10 0.0000e+00 0.0000e+00] 0 0 1 -1 1 0.0 3.5668e-10
[0.0000e+00 3.5668e-10 0.0000e+00] 0 0 1 1 -1 0.0 3.5668e-10
[-2.6751e-10 -2.6751e-10 -8.9170e-11] 0 1 -1 0 0 102.15417881222723 3.8868301879552184e-10
[ 8.9170e-11 -2.6751e-10  2.6751e-10] 0 1 -1 0 2 61.29250728733634 3.8868301879552184e-10
[-2.6751e-10  8.9170e-11  2.6751e-10] 0 1 -1 2 0 61.29250728733634 3.8868301879552184e-10
[-8.9170e-11 -2.6751e-10 -2.6751e-10] 0 1 0 -1 0 102.15417881222723 3.8868301879552184e-10
[ 2.6751e-10 -2.6751e-10  8.9170e-11] 0 1 0 -1 2 61.29250728733634 3.8868301879552184e-10
[-2.6751e-10 -8.9170e-11 -2.6751e-10] 0 1 0 0 -1 102.15417881222723 3.8868301879552184e-10
[ 2.6751e-10 -8.9170e-11  2.6751e-10] 0 1 0 0 2 20.43083576244544 3.88683018795

np.float64(3.8831173044346727e-10)

# State reconstruction

In [4]:
@jit
def index_to_coord(index, max_distance, site_nb):
    center = max_distance // 2
    return (
        index // (max_distance**2 * site_nb**2) - center,
        index // (max_distance * site_nb**2) % max_distance - center,
        index // site_nb**2 % max_distance - center,
        index // site_nb % site_nb,
        index % site_nb
    )  

@jit
def coord_to_index(vec, max_distance, site_nb):
    center = max_distance // 2
    return site_nb**2 * ((vec[0] + center)*max_distance**2 + (vec[1] + center) * max_distance + (vec[2] + center)) + vec[3]*site_nb + vec[4]

def vector_couplings(max_distance, site_nb, tolerance):
    print("Computing vector -> couplings lookup")
    couplings = np.empty(max_distance**3*site_nb**2)
    vectors = np.empty((max_distance**3*site_nb**2, 5), dtype=np.int64)
    for i in range(max_distance**3*site_nb**2):
        vec = index_to_coord(i, max_distance, site_nb)
        couplings[i] = compute_coupling(
                    *vec
                )
        vectors[i] = np.array(vec)
    sorted_couplings = np.argsort(couplings)
    print("Successfully reverted the map")
    return couplings[sorted_couplings], vectors[sorted_couplings]

In [26]:
@jit
def cost_function(sites, couplings):
    couplings_theory = couplings(sites)
    cost = 0
    for i in range(sites.shape[0]-1):
        for j in range(i+1, sites.shape[0]):
            if np.isnan(couplings[i,j]):
                continue
            x = sites[i][0] - sites[j][0]
            y = sites[i][1] - sites[j][1]
            z = sites[i][2] - sites[j][2]
            cost += (couplings[i,j] - compute_coupling(x, y, z, sites[i][3], sites[j][3]))**2
    return cost

@jit
def exchange_columns(couplings, permutation, a, b):
    a, b = min(a, b), max(a, b)
    print(f"Exchange {a}-{b}")
    permutation[a], permutation[b] = permutation[b], permutation[a]
    for i in range(a):
        couplings[i, a], couplings[i, b] = couplings[i, b], couplings[i, a]
    for i in range(a+1, b):
        couplings[a, i], couplings[i, b] = couplings[i, b], couplings[a, i]
    for i in range(b+1, couplings.shape[0]): 
        couplings[a, i], couplings[b, i] = couplings[b, i], couplings[a, i]

def set_placing_order(couplings):
    print(couplings.shape)
    n_tot = couplings.shape[0]
    permutation = np.arange(n_tot)
    m = np.nanmax(np.abs(couplings[0,:]))
    first = 0
    for i in range(1, n_tot):
        if np.nanmax(np.abs(couplings[i,:]))> m:
            m = np.nanmax(np.abs(couplings[i,:]))
            first = i
    exchange_columns(couplings, permutation, 0, first)
    for i in range(1, n_tot):
        next_index = np.nanargmax(np.abs(couplings[:i,i:]))%(n_tot-i) + i
        if next_index != i:
            exchange_columns(couplings, permutation, i, next_index)
    return couplings, permutation

@jit
def compute_new_possible_config(possible_configurations, current_couplings, candidates, tolerance, edge_spin, n_placed):
    new_possible_configurations = []
    for config in possible_configurations:
        for candidate in candidates:
            good_candidate = 1
            # First check the origin lattice site
            if candidate[4] != config[edge_spin][3]:
                continue
            for i in range(n_placed-1, -1, -1):
                if np.isnan(current_couplings[i]):
                    # no data for this already placed spin
                    continue
                x = candidate[0] + config[edge_spin][0] - config[i][0]
                y = candidate[1] + config[edge_spin][1] - config[i][1]
                z = candidate[2] + config[edge_spin][2] - config[i][2]
                a = candidate[3]
                b = config[i][3]
                if (x, y, z) == (0,0,0) and a == b:
                    good_candidate = False
                    break
                coupling_candidate = compute_coupling(
                    x, y, z, a, b
                )
                if np.isnan(coupling_candidate) or np.abs(current_couplings[i] - coupling_candidate) > tolerance:
                    good_candidate = False
                    break
            if good_candidate:
                new_config = config.copy()
                new_config[n_placed] = np.array((candidate[0] + config[edge_spin][0], candidate[1] + config[edge_spin][1], candidate[2] + config[edge_spin][2], candidate[3]))
                new_possible_configurations.append(new_config)
    return new_possible_configurations

@jit
def all_error_cost(configs, coupl, n_max):
    errors = np.empty(len(configs))
    for i, config in enumerate(configs):
        sim_couplings = couplings(config)
        for i in range(n_max,27):
            for j in range(27):
                sim_couplings[j,i] = np.nan
        errors[i] = np.nansum((sim_couplings-coupl)**2)
    return errors

#@jit
def compute_sites(couplings, lattice_size, site_nb, tolerance):
    print("Begin")
    n_placed = 1
    n_tot = couplings.shape[0]
    couplings, permutation = set_placing_order(couplings)
    print("ordered spins")
    couplings_vectors_tup = vector_couplings(tolerance = tolerance, max_distance = lattice_size*2, site_nb = site_nb)
    print("computed lookup table")
    possible_configurations = []
    for i in range(site_nb):
        c = np.zeros((n_tot, 4), dtype = np.int64)
        c[0][3] = i
        possible_configurations.append(c)
        
    current_couplings = np.empty(n_tot) # stores current coupling values to other spins

    print("Initialization successful")
    while n_placed < n_tot:

        for i in range(n_tot):
            current_couplings[i] = couplings[n_placed, i] if np.isnan(couplings[i, n_placed]) else couplings[i, n_placed]
        # Be careful, position relative to edge_spin

        edge_spin = np.nanargmax(current_couplings[:n_placed])

        # Get candidates
        candidate_min = np.searchsorted(couplings_vectors_tup[0], current_couplings[edge_spin]-tolerance, side = 'left')
        candidate_max = np.searchsorted(couplings_vectors_tup[0], current_couplings[edge_spin]+tolerance, side = 'right')
        candidates = couplings_vectors_tup[1][candidate_min:candidate_max]
        print(f"Placing {n_placed} (linked to {edge_spin}). {current_couplings[edge_spin]} Hz. {len(candidates)} for each one of the {len(possible_configurations)} cases to process.")

        new_possible_configurations = compute_new_possible_config(possible_configurations, current_couplings, candidates, tolerance, edge_spin, n_placed)
        
        if len(new_possible_configurations) == 0:
            print("Ending prematurely")
            return possible_configurations, permutation
        if len(new_possible_configurations) > 5000:
            argsort_error = np.argsort(all_error_cost(new_possible_configurations, couplings, n_placed + 1))
            possible_configurations = [new_possible_configurations[argsort_error[i]] for i in range(5000)]
        else:
            possible_configurations = new_possible_configurations
        n_placed+=1
    return possible_configurations, permutation



In [27]:
def check_configuration(sites, original):
    """
    Check if configurations are the same up to a translation
    """
    x_sites = np.min(sites[:,0])
    x_original = np.min(original[:,0])
    y_sites = np.min(sites[:,1])
    y_original = np.min(original[:,1])
    z_sites = np.min(sites[:,2])
    z_original = np.min(original[:,2])

    # useless allocation here we could do it in place...
    sites = sites + np.array([[x_original - x_sites, y_original - y_sites, z_original - z_sites, 0]])
    sites = np.sort(sites, axis=0)
    original = np.sort(original, axis=0)
    return np.array_equal(sites, original)

    
    

###  Load data

In [28]:
raw_data = np.genfromtxt("diamond_couplings.csv", delimiter=";", filling_values = np.nan)

renormalized_data = np.empty(raw_data[1:,1:].shape)
for i in range(1,raw_data.shape[0]):
    for j in range(1,raw_data.shape[1]):
        if np.isclose(raw_data[i, j], 0, rtol=0):
            renormalized_data[i-1, j-1] = np.nan
        else:
            renormalized_data[i-1, j-1] = raw_data[i, j]


print("Measured couplings")
pd.DataFrame(renormalized_data)


Measured couplings


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NaN,61.908,62.188,235.92,8.577,12.687,5.420,2.820,1.120,NaN,...,1.120,NaN,NaN,2.540,NaN,NaN,7.720,2.02,NaN,NaN
1,NaN,NaN,235.777,62.12,19.910,19.227,5.920,1.920,1.068,2.110,...,1.020,NaN,NaN,2.879,NaN,NaN,4.710,3.72,1.260,NaN
2,NaN,NaN,NaN,25.11,5.210,26.909,20.130,12.688,2.506,NaN,...,0.995,NaN,NaN,2.020,NaN,NaN,5.468,4.82,3.120,NaN
3,NaN,NaN,NaN,NaN,19.630,7.120,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1.870,1.360,1.320,1.167,8.827,...,1.820,0.820,7.52,12.609,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,16.597,12.378,6.613,NaN,...,1.186,0.710,1.13,1.820,NaN,0.82,3.388,4.52,4.998,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.437,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.595,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.220,...,1.274,9.012,NaN,NaN,3.767,0.82,NaN,NaN,3.310,0.924
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.720,6.92,3.820,NaN,NaN,NaN,NaN,NaN,NaN


### Compute

In [ ]:
import time

t0 = time.perf_counter()
final_sites, permutation = compute_sites(renormalized_data, lattice_size = 11, site_nb=site_nb, tolerance = 3)
t = time.perf_counter() - t0
print(f"Time {t}")


Begin
(27, 27)
Exchange 0-0
Exchange 1-3
Exchange 4-5
Exchange 5-6
Exchange 8-20
Exchange 11-19
Exchange 12-23
Exchange 13-20
Exchange 14-18
Exchange 16-18
Exchange 17-20
Exchange 19-26
Exchange 20-25
Exchange 21-24
Exchange 22-24
Exchange 24-26
ordered spins
Computing vector -> couplings lookup


/tmp/ipykernel_777111/3237595639.py:34: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(np.abs(couplings[i,:]))> m:
/nix/store/29g15dxg5v3z5zqmbi5mssak05h9k0l4-python3-3.12.9-env/lib/python3.12/site-packages/numba/np/ufunc/dufunc.py:288: RuntimeWarning: divide by zero encountered in compute_coupling
  return super().__call__(*args, **kws)
/nix/store/29g15dxg5v3z5zqmbi5mssak05h9k0l4-python3-3.12.9-env/lib/python3.12/site-packages/numba/np/ufunc/dufunc.py:288: RuntimeWarning: invalid value encountered in compute_coupling
  return super().__call__(*args, **kws)


Successfully reverted the map
computed lookup table
Initialization successful
Placing 1 (linked to 0). 235.92 Hz. 24 for each one of the 2 cases to process.
Placing 2 (linked to 0). 62.188 Hz. 12 for each one of the 24 cases to process.
Placing 3 (linked to 2). 235.777 Hz. 24 for each one of the 36 cases to process.
Placing 4 (linked to 2). 26.909 Hz. 60 for each one of the 12 cases to process.
Placing 5 (linked to 2). 20.13 Hz. 60 for each one of the 12 cases to process.
Placing 6 (linked to 3). 19.91 Hz. 60 for each one of the 24 cases to process.
Placing 7 (linked to 2). 12.688 Hz. 216 for each one of the 24 cases to process.
Placing 8 (linked to 6). 12.609 Hz. 222 for each one of the 72 cases to process.
Placing 9 (linked to 6). 8.827 Hz. 362 for each one of the 3888 cases to process.
Placing 10 (linked to 9). 14.487 Hz. 152 for each one of the 5000 cases to process.
Placing 11 (linked to 10). 9.52 Hz. 342 for each one of the 5000 cases to process.
Placing 12 (linked to 0). 7.72 Hz

In [ ]:
@jit
def error_cost(sim_couplings, permutation, renormalized_data):
    for i in range(22,27):
        for j in range(27):
            sim_couplings[j,i] = np.nan
    permuted_couplings = np.full((27,27), np.nan)
    for i in range(26):
        for j in range(i+1, 27):
            a, b = min(permutation[i], permutation[j]), max(permutation[i], permutation[j])
            permuted_couplings[a, b] = sim_couplings[i, j]
    return np.nansum((permuted_couplings-renormalized_data)**2)
    


m = np.inf
sol_min = -1
print("len:", len(final_sites))
for sol in range(len(final_sites)):
    sim_couplings = couplings(final_sites[sol])
    n = error_cost(sim_couplings, permutation, renormalized_data)
    if n < m:
        sol_min = sol
        m = n
    print(sol, end="\r")

In [ ]:
sol_min

In [ ]:
# https://stackoverflow.com/questions/11649577/how-to-invert-a-permutation-array-in-numpy
def invert_permutation(p):
    """Return an array s with which np.array_equal(arr[p][s], arr) is True.
    The array_like argument p must be some permutation of 0, 1, ..., len(p)-1.
    """
    p = np.asanyarray(p) # in case p is a tuple, etc.
    s = np.empty_like(p)
    s[p] = np.arange(p.size)
    return s



In [ ]:
%matplotlib widget

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')

permutation_inv = invert_permutation(permutation.copy())

sites = final_sites[sol_min]

positions_raw = lattice_x[None,:]*sites[:,0][:,None] + lattice_y[None,:] * sites[:,1][:,None] + lattice_z[None,:] * sites[:,2][:,None] + lattice_s[sites[:,3]]

positions_raw -= positions_raw[permutation_inv[0]]

from scipy.spatial.transform import Rotation as R

first_rotation = R.from_rotvec(-np.arccos((2/3)**0.5)*np.array([-1, 1, 0]))

positions = first_rotation.apply(positions_raw)


ax.scatter(positions[:,0], positions[:,1], positions[:,2])


for i in range(27):
    ax.text(positions[permutation_inv[i],0], positions[permutation_inv[i],1], positions[permutation_inv[i],2], str(i+1), color="orange")

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.axis('equal')

In [22]:
len(final_sites)

5000